In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import glob, pylab, pandas as pd
import pydicom, numpy as np
from os import listdir
import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F

from os.path import isfile, join
import matplotlib.pylab as plt
import os
import seaborn as sns
import warnings
warnings.filterwarnings(action='once')
import pickle
from tqdm import tqdm, tqdm_notebook
%load_ext autoreload
%autoreload 2
%matplotlib notebook
from helper import *
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from defenitions import *

/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
data_dir = '/media/nvme/data/RSNA/'
train_images_dir=data_dir+'stage_1_train_images/'
test_images_dir=data_dir+'stage_2_test_images/'

In [3]:
train_base_df = pd.read_csv(data_dir+'stage_2_train.csv')

In [4]:
train_base_df.shape[0]

4516842

In [5]:
train_base_df['Sub_type'] = train_base_df['ID'].str.split("_", n = 3, expand = True)[2]
train_base_df['PatientID'] = train_base_df['ID'].str.split("_", n = 3, expand = True)[1]
train_base_df.head()

,ID,Label,Sub_type,PatientID
0,ID_12cadc6af_epidural,0,epidural,12cadc6af
1,ID_12cadc6af_intraparenchymal,0,intraparenchymal,12cadc6af
2,ID_12cadc6af_intraventricular,0,intraventricular,12cadc6af
3,ID_12cadc6af_subarachnoid,0,subarachnoid,12cadc6af
4,ID_12cadc6af_subdural,0,subdural,12cadc6af


In [6]:
train_base_df=train_base_df.drop_duplicates().reset_index(drop=True)

In [7]:
assert train_base_df.PatientID.unique().shape[0]==train_base_df.shape[0]//6

In [8]:
sub_types=train_base_df.Sub_type.unique()
sub_types

array(['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid',
       'subdural', 'any'], dtype=object)

In [9]:
dfs =[]
for sub_type in tqdm_notebook(sub_types):
    df = train_base_df[train_base_df.Sub_type==sub_type][['PatientID','Label']].copy()
    df=df.rename(columns={"Label": sub_type}).reset_index(drop=True)
    dfs.append(df)
train_df=dfs[0]
for df in tqdm_notebook(dfs[1:]):
    train_df=train_df.merge(df,on='PatientID')

In [10]:
assert train_df.shape[0]==train_base_df.shape[0]//6

In [11]:
train_df=train_df[~train_df.PatientID.isin(bad_images)]

In [12]:
df1 = pd.read_csv(data_dir+'train_dedup.csv')
df2 = pd.read_csv(data_dir+'test.csv')
old_dfs = pd.concat([df1,df2],ignore_index=True,sort=False).drop(hemorrhage_types,axis=1)
old_dfs.head()
old_dfs.shape
assert old_dfs.shape[0]==train_df.shape[0]

,PatientID,PID,StudyI,SeriesI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,63eb1e259,a449357f,62d125e5b2,0be5c0d1b3,"['00036', '00036']","['00080', '00080']",180.199951,-125.0,-8.000000
1,2669954a7,363d5865,a20b80c7bf,3564d584db,"['00047', '00047']","['00080', '00080']",922.530821,-156.0,45.572849
2,52c9913b1,9c2b4bd7,3e3634f8cf,973274ffc9,40,150,4.455000,-125.0,-115.063000
3,4e6ff6126,3ae81c2d,a1390c15c2,e5ccad8244,"['00036', '00036']","['00080', '00080']",100.000000,-99.5,28.500000
4,7858edd88,c1867feb,c73e81ed3a,28e0531b3a,40,100,145.793000,-125.0,-132.190000


(752797, 9)

In [13]:
train_df=train_df.merge(old_dfs,on='PatientID')
train_df.head()
assert old_dfs.shape[0]==train_df.shape[0]

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,PID,StudyI,SeriesI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,12cadc6af,0,0,0,0,0,0,2eb3925f,6dec708c74,1b17a4a944,30,80,71.652000,-125.0,-145.337000
1,38fd7baa0,0,0,0,0,0,0,e18d3063,0a9ac70962,3f92acee54,30,80,226.174000,-125.0,-140.052000
2,6c5d82413,0,0,0,0,0,0,dbb88395,45e4c06199,5467fe8129,"['00036', '00036']","['00080', '00080']",-199.400009,-115.0,-22.000000
3,aec8e68b3,0,0,0,1,0,1,dbfa26fc,0d739f7065,e01d9cd8c9,30,80,98.205338,-125.0,-109.982063
4,4d9209c7c,0,0,0,0,0,0,ce982e0b,f361229319,04b809de14,"['00036', '00036']","['00080', '00080']",280.300049,-125.0,-8.000000


In [35]:
train_df.to_csv(data_dir+'train_stage2.csv',index=False)

In [14]:
old_base_test=pd.read_csv(data_dir+'stage_1_sample_submission.csv')
old_test_ids=set(old_base_test.ID.values)
old_test_res=train_base_df[train_base_df.ID.isin(old_test_ids)]
old_test_res.shape
old_test_res.to_csv(data_dir+'true_submission_stage1.csv',index=False)

(471270, 4)

In [6]:
sample_submission=pd.read_csv(data_dir+'stage_2_sample_submission.csv')
sample_submission.head()
sample_submission.shape

,ID,Label
0,ID_0fbf6a978_epidural,0.5
1,ID_0fbf6a978_intraparenchymal,0.5
2,ID_0fbf6a978_intraventricular,0.5
3,ID_0fbf6a978_subarachnoid,0.5
4,ID_0fbf6a978_subdural,0.5


(727392, 2)

In [16]:
test_base_df=sample_submission.copy()
test_base_df['Sub_type'] = test_base_df['ID'].str.split("_", n = 3, expand = True)[2]
test_base_df['PatientID'] = test_base_df['ID'].str.split("_", n = 3, expand = True)[1]
test_base_df.head()

,ID,Label,Sub_type,PatientID
0,ID_0fbf6a978_epidural,0.5,epidural,0fbf6a978
1,ID_0fbf6a978_intraparenchymal,0.5,intraparenchymal,0fbf6a978
2,ID_0fbf6a978_intraventricular,0.5,intraventricular,0fbf6a978
3,ID_0fbf6a978_subarachnoid,0.5,subarachnoid,0fbf6a978
4,ID_0fbf6a978_subdural,0.5,subdural,0fbf6a978


In [17]:
assert test_base_df.shape[0] == test_base_df.PatientID.unique().shape[0]*6

In [18]:
sub_types=test_base_df.Sub_type.unique()
sub_types

array(['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid',
       'subdural', 'any'], dtype=object)

In [19]:
dfs =[]
for sub_type in tqdm_notebook(sub_types):
    df = test_base_df[test_base_df.Sub_type==sub_type][['PatientID','Label']].copy()
    df=df.rename(columns={"Label": sub_type}).reset_index(drop=True)
    dfs.append(df)
test_df=dfs[0]
for df in tqdm_notebook(dfs[1:]):
    test_df=test_df.merge(df,on='PatientID')

In [20]:
test_ids=test_df.PatientID.unique()

In [21]:
test_df.head()

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any
0,0fbf6a978,0.5,0.5,0.5,0.5,0.5,0.5
1,d62ec3412,0.5,0.5,0.5,0.5,0.5,0.5
2,cb544194b,0.5,0.5,0.5,0.5,0.5,0.5
3,0d62513ec,0.5,0.5,0.5,0.5,0.5,0.5
4,fc45b2151,0.5,0.5,0.5,0.5,0.5,0.5


In [22]:
PID = np.zeros(test_df.shape[0],dtype=object)
StudyI = np.zeros(test_df.shape[0],dtype=object)
SeriesI = np.zeros(test_df.shape[0],dtype=object)
WindowCenter = np.zeros(test_df.shape[0],dtype=object)
WindowWidth = np.zeros(test_df.shape[0],dtype=object)
ImagePositionX = np.zeros(test_df.shape[0],dtype=np.float)
ImagePositionY = np.zeros(test_df.shape[0],dtype=np.float)
ImagePositionZ = np.zeros(test_df.shape[0],dtype=np.float)

for i,row in tqdm_notebook(test_df.iterrows(),total=test_df.shape[0]):
    ds = pydicom.dcmread(test_images_dir + 'ID_{}.dcm'.format(row['PatientID']))
    SeriesI[i]=ds.SeriesInstanceUID
    PID[i]=ds.PatientID
    StudyI[i]=ds.StudyInstanceUID 
    WindowCenter[i]=ds.WindowCenter
    WindowWidth[i]=ds.WindowWidth
    ImagePositionX[i]=float(ds.ImagePositionPatient[0])
    ImagePositionY[i]=float(ds.ImagePositionPatient[1])
    ImagePositionZ[i]=float(ds.ImagePositionPatient[2])
test_df['SeriesI']=SeriesI
test_df['PID']=PID
test_df['StudyI']=StudyI
test_df['WindowCenter']=WindowCenter
test_df['WindowWidth']=WindowWidth
test_df['ImagePositionZ']=ImagePositionZ
test_df['ImagePositionX']=ImagePositionX
test_df['ImagePositionY']=ImagePositionY


In [25]:
test_df['SeriesI'] = test_df['SeriesI'].str.split("_", n = 3, expand = True)[1]
test_df['PID'] = test_df['PID'].str.split("_", n = 3, expand = True)[1]
test_df['StudyI'] = test_df['StudyI'].str.split("_", n = 3, expand = True)[1]


In [4]:
test_df.head()

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,SeriesI,PID,StudyI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,0fbf6a978,0.5,0.5,0.5,0.5,0.5,0.5,48eaa460c6,3556ae0e,3710aff1c5,40,100,37.690857,-125.0,-117.830185
1,d62ec3412,0.5,0.5,0.5,0.5,0.5,0.5,389cec2e11,c667d610,8e38d86ede,"['00036', '00036']","['00080', '00080']",762.200043,-66.0,-10.000000
2,cb544194b,0.5,0.5,0.5,0.5,0.5,0.5,a6ebe3ca4f,9389d3b6,7040866911,"['00036', '00036']","['00080', '00080']",818.400024,-105.0,6.000000
3,0d62513ec,0.5,0.5,0.5,0.5,0.5,0.5,01c388cfe3,79cdc6e4,07c298b5be,30,80,31.857132,-125.0,-112.984024
4,fc45b2151,0.5,0.5,0.5,0.5,0.5,0.5,d16ad143cb,39eb6eb6,6a15b35eae,"['00036', '00036']","['00080', '00080']",424.099976,-102.5,14.500000


In [27]:
test_df.to_csv(data_dir+'test_stage2.csv',index=False)

In [3]:
test_df=pd.read_csv(data_dir+'test_stage2.csv')

In [5]:
pickle_file=open(outputs_dir+'test_indexes_stage2.pkl','wb')
pickle.dump(test_df.PatientID.values,pickle_file,protocol=4)
pickle_file.close()